In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

### bank_termdeposit.csv
- 해당 파일은 Kaggle에서 구한 자료입니다.
- 고객이 정기예금을 가입하기 위해 어느 변수가 주요하게 작용하는지를 알아보는 data set 입니다.  

### Data 설명
#### 고객 Data
- 1. age (numeric)
- 2. job : type of job (categorical: "admin.", "unknown", "unemployed", "management", "housemaid", "entrepreneur", "student", "blue-collar", "self-employed", "retired", "technician", "services")
- 3. marital : marital status (categorical: "married", "divorced", "single"; note: "divorced" means divorced or widowed)
- 4. education (categorical: "unknown", "secondary", "primary", "tertiary")
- 5. default: has credit in default? (binary: "yes", "no")
- 6. balance: average yearly balance (입출금 계좌), in euros (numeric)
- 7. housing: has housing loan? (binary: "yes", "no")
- 8. loan: has personal loan? (binary: "yes", "no")

#### 직전 campaign 을 통한 고객과의 접점 관련 정보
- 9. contact: contact communication type (categorical: "unknown", "telephone", "cellular")
- 10. day: last contact day of the month (numeric)
- 11. month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")
- 12. duration: last contact duration, in seconds (numeric)

#### 기타 속성
- 13. campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
- 14. pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted)
- 15. previous: number of contacts performed before this campaign and for this client (numeric)
- 16. poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")

#### 종속 변수 (목표 target):
- 17. subscribed - has the client subscribed a term deposit? (binary: "yes","no")

In [15]:
bank = pd.read_csv("data/bank_termdeposit.csv")

# bank_termdiposit 파일에 할당된 데이터의 행렬 사이즈를 출력
print(bank.shape)

# bank_termdeposit 데이터의 상위 5 row
bank.head()

(31647, 18)


,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,subscribed
0,26110,56,admin.,married,unknown,no,1933,no,no,telephone,19,nov,44,2,-1,0,unknown,no
1,40576,31,unknown,married,secondary,no,3,no,no,cellular,20,jul,91,2,-1,0,unknown,no
2,15320,27,services,married,secondary,no,891,yes,no,cellular,18,jul,240,1,-1,0,unknown,no
3,43962,57,management,divorced,tertiary,no,3287,no,no,cellular,22,jun,867,1,84,3,success,yes
4,29842,31,technician,married,secondary,no,119,yes,no,cellular,4,feb,380,1,-1,0,unknown,no


In [17]:
# 주요 통계지표 확인
bank.describe()

,ID,age,balance,day,duration,campaign,pdays,previous
count,31647.000000,31647.000000,31647.000000,31647.000000,31647.000000,31647.000000,31647.000000,31647.000000
mean,22563.972162,40.957247,1363.890258,15.835466,258.113534,2.765697,39.576042,0.574272
std,13075.936990,10.625134,3028.304293,8.337097,257.118973,3.113830,99.317592,2.422529
min,2.000000,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,11218.000000,33.000000,73.000000,8.000000,104.000000,1.000000,-1.000000,0.000000
50%,22519.000000,39.000000,450.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,33879.500000,48.000000,1431.000000,21.000000,318.500000,3.000000,-1.000000,0.000000
max,45211.000000,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


In [30]:
# 위에 출력된 변수들은 numeric 값을 담은 변수로 DataFrame에 활용하기 위해 따로 columns 리스트를 저장.
bank_numeric = bank.describe()
bank_numeric.columns

Index(['ID', 'age', 'balance', 'day', 'duration', 'campaign', 'pdays',
       'previous'],
      dtype='object')

#### numeric 값을 포함한 독립변수들에 대한 해석

In [32]:
# 종속변수 (subscribed 변수)의 값에 따라 각 독립변수 중 numeric 값을 포함한 변수들의 평균값 비교
# ID는 통계를 내는 것이 무의미하므로 제외함.

pd.pivot_table(bank, index = 'subscribed', values = bank_numeric.columns.drop("ID"))

,age,balance,campaign,day,duration,pdays,previous
subscribed,,,,,,,
no,40.862165,1307.779822,2.845876,15.925462,221.559108,35.653802,0.496456
yes,41.672140,1785.768237,2.162853,15.158816,532.955585,69.066218,1.159354


#### 위의 변수들에 대한 해석
**아래의 해석은 단순히 위의 표의 결과를 토대로 한 해석입니다. 선형회귀분석 등의 추가 검증은 추후 진행 예정이며, 추가 검증 결과 다르게 해석 될 수도 있습니다**
- age : 나이는 정기예금 가입 여부에 따라 큰 차이를 보이지 않으므로 큰 영향이 없는 것으로 보입니다.
- balance : 정기예금을 가입 안한 이들의 평균 잔액이 가입한 고객들의 잔액보다 현저히 작은 것을 확인할 수 있으므로 정기예금 가입 여부와 상관관계가 있는 것으로 보임. 즉, 여유 잔액이 많은 고객일 수록 정기예금을 가입하는 경향이 높다라고 해석해 볼 수도 있습니다.
- campaign : 금번 캠페인 기간 동안 고객과 접촉한 수를 나타내며, 오히려 정기예금을 가입하지 않은 고객들이 평균적으로 캠페인 기간 동안 은행과의 접촉 수가 많은 것으로 확인됨. 이를 귀납적으로 추론해보자면, 정기예금에 가입하지 않으려는 경향이 보였기에 은행에서 캠페인 기간동안 더 많은 횟수의 접촉을 시도했다고도 볼 수 있을 것입니다.
- day : 마지막 접촉한 일자를 나타내는 지표로 평균적으로 큰 차이를 보이지 않는 것으로 보아 큰 영향이 없는 것으로 보입니다.
- duration : 마지막으로 고객과 접촉 당시 얼마나 오래 접촉하였는지(단위 : 초)를 나타내는 지표로, 정기예금에 가입한 고객들이 평균적으로 더 긴 시간동안 고객과 접촉한 것으로 확인됩니다. 이는 접촉하는 당시 고객이 캠페인에 긍정적으로 반응하였기에 더 긴 시간 접촉하였다고도 볼 수 있을 것 같습니다.
- pdays : 고객과 접촉한 마지막 일자로부터 경과된 시간(단위 : 일)을 나타내는 지표로 정기예금에 가입하지 않은 고객들이 평균적으로 비교적 최근에 접촉이 된 것으로 확인됩니다. 이는 campain 변수와 유사한 논리인 정기예금에 가입하지 않으려는 경향을 보였기에 은행에서 비교적 최근까지도 접촉을 시도한 것으로 해석해볼 수 있습니다.
- previous : 금번이 아닌 이전 campaign 기간 동안 고객과의 접촉한 횟수를 나타냅니다. 평균적으로 정기예금에 가입한 고객들이 가입하지 않은 고객들보다 은행과 많은 접촉이 있는 것으로 확인됩니다. 이 부분이 개인적으로 조금 흥미로웠는데, 이는 앞서 살펴본 campaign 변수에서 살펴본 해석과 같이 연결지어볼 필요가 있을 것 같습니다. 추가적인 검증이 필요하지만 제가 우선 위의 pivot_table 만으로 귀납추론을 하면 다음과 같습니다. 정기예금에 가입하지 않으려는 고객에 은행은 평균적으로 보다 더 많은 접촉을 시도합니다. 이에 가입을 할 수도, 안 할 수도 있으며 결국 가입을 안 한 고객들은 평균적으로 높은 접촉 횟수를 나타내게 됩니다. 그러나 이 previous 의 결과를 살펴보면, 직전 campaign에서 접촉을 많이 했던 고객들은 결국 다음 campaign 을 거치면서 가입하는 경향을 보이는 것 같습니다. 즉, campaign의 접촉 횟수는 비록 처음에는 많더라도 고객유치에 실패할 수 있으나, 결국 이후의 campaign 에는 고객을 유치하는데 성공하게 만드는 요인이 되는 것 같습니다. 

#### 문자열 값을 포함한 독립변수들에 대한 해석
- 'job', 'marital', 'education', 'contact'

**```"job"```으로 우선 pivot_table 생성**

In [81]:
# 우선, 정기예금을 가입한 사람들만 추출
bank_yes = bank[bank["subscribed"] == 'yes']

In [82]:
# 직업별 pivot_table 생성
bank_job = pd.pivot_table(bank, index = 'job', values = 'ID', aggfunc = 'count')
bank_job.head()

,ID
job,
admin.,3631
blue-collar,6842
entrepreneur,1008
housemaid,874
management,6639


In [83]:
# 정기에금 가입한 사람들 중 직업별 pivot_table 생성
bank_yes_job = pd.pivot_table(bank_yes, index = 'job', values = 'ID', aggfunc = 'count')
bank_yes_job.head()

,ID
job,
admin.,452
blue-collar,489
entrepreneur,85
housemaid,79
management,923


In [84]:
bank_yes_job = pd.DataFrame(round(bank_yes_job["ID"] / bank_job["ID"]  * 100))
bank_yes_job.rename(columns = {"ID" : "subscribed (%)"}, inplace = True)
bank_yes_job

,subscribed (%)
job,
admin.,12.0
blue-collar,7.0
entrepreneur,8.0
housemaid,9.0
management,14.0
retired,23.0
self-employed,12.0
services,9.0
student,29.0


- 위의 방식으로 나머지 변수들도 계산 : ```"marital"```, ```"education"```, ```"contact"```

In [85]:
# 정기예금 가입여부 상관없이 모든 데이터에 대한 pivot_table 생성
bank_marital = pd.pivot_table(bank, index = 'marital', values = 'ID', aggfunc = 'count')
bank_education = pd.pivot_table(bank, index = 'education', values = 'ID', aggfunc = 'count')
bank_contact = pd.pivot_table(bank, index = 'contact', values = 'ID', aggfunc = 'count')

# 정기에금 가입한 사람들 대상 pivot_table 생성
bank_yes_marital = pd.pivot_table(bank_yes, index = 'marital', values = 'ID', aggfunc = 'count')
bank_yes_education = pd.pivot_table(bank_yes, index = 'education', values = 'ID', aggfunc = 'count')
bank_yes_contact = pd.pivot_table(bank_yes, index = 'contact', values = 'ID', aggfunc = 'count')

# percentage 계산
bank_yes_marital = pd.DataFrame(round(bank_yes_marital["ID"] / bank_marital["ID"]  * 100))
bank_yes_marital.rename(columns = {"ID" : "subscribed (%)"}, inplace = True)

bank_yes_education = pd.DataFrame(round(bank_yes_education["ID"] / bank_education["ID"]  * 100))
bank_yes_education.rename(columns = {"ID" : "subscribed (%)"}, inplace = True)

bank_yes_contact = pd.DataFrame(round(bank_yes_contact["ID"] / bank_contact["ID"]  * 100))
bank_yes_contact.rename(columns = {"ID" : "subscribed (%)"}, inplace = True)

In [94]:
# bank_marital
bank_yes_marital

,subscribed (%)
marital,
divorced,12.0
married,10.0
single,15.0


In [95]:
# bank_education
bank_yes_education

,subscribed (%)
education,
primary,9.0
secondary,10.0
tertiary,15.0
unknown,13.0


In [96]:
bank_yes_contact

,subscribed (%)
contact,
cellular,15.0
telephone,13.0
unknown,4.0


#### 위의 변수들에 대한 해석
**아래의 해석은 단순히 위의 표의 결과를 토대로 한 해석입니다. 선형회귀분석 등의 추가 검증은 추후 진행 예정이며, 추가 검증 결과 다르게 해석 될 수도 있습니다**
- job : 학생과 은퇴한 고객들이 평균적으로 정기예금을 많이 가입하는 경향이 나타남.
- marital : 결혼한 고객보다 싱글 혹은 이혼한 고객들이 평균적으로 정기예금을 많이 가입하는 경향이 나타남.
- education : 초등, 중등, 고등학교 학력을 비교해볼 때 학력이 높은 고객일 수록 평균적으로 정기예금을 많이 가입하는 경향이 나타남.
- contact : 고객의 핸드폰을 통해 접촉할 수록 고객들이 평균적으로 정기예금을 많이 가입하는 경향이 나타남.